In [49]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import nltk
import re
import string
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.externals import joblib
from Function_file import *

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vishnu.raju\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\vishnu.raju\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vishnu.raju\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\vishnu.raju\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [17]:
#Data
categories = pd.read_csv('Catagory.csv')
categories = categories.loc[:,['ID','NAME']]
categories.columns = ['CATEGORY_ID','NAME']
categories.head()

,CATEGORY_ID,NAME
0,102786,Document Binders
1,102787,Magnets
2,102788,Plasma
3,102789,Whole Blood
4,102790,"Research Tissues, Organs and Glands"


In [18]:
items = pd.read_csv('Item_Filtered.csv')
items.head()

,ID,CATEGORY_ID,BATCH_REFERENCE,DESCRIPTION,LONG_DESCRIPTION,MANF_NAME
0,9683165,106733,Office Depot526G8183NewItems-CatagoeryDeselece...,FLUTE OUTFIT ODYSSEY,<p><b>VAT Rate =20%<b/><p/>,Office Depot
1,202859,101708,OfficeDepot526G6450CambridgeFurnitureHides2011...,Oki C9200/C9400 Toner Cartridge 41515210 Magenta,Oki C9000 Toner Cartridge Magenta 41515210<p><...,Oki
2,9683166,109322,Office Depot526G8183NewItems-CatagoeryDeselece...,CLARINET OUTFIT ODYSSEY,<p><b>VAT Rate =20%<b/><p/>,Office Depot
3,202858,101708,OfficeDepot526G6450CambridgeFurnitureHides2011...,Oki C9200/C9400 Toner Cartridge 41515211 Cyan,Oki C9000 Toner Cartridge Cyan 41515211<p><b>V...,Oki
4,9683167,109322,Office Depot526G8183NewItems-CatagoeryDeselece...,ALTO SAX OUTFIT ODYSSEY,<p><b>VAT Rate =20%<b/><p/>,Office Depot


In [30]:
#Final data
data = items.merge(categories,left_on='CATEGORY_ID',right_on='CATEGORY_ID',how='left')
data['text'] = data.DESCRIPTION + ' ' + data.MANF_NAME
data = data.drop(columns=['BATCH_REFERENCE','DESCRIPTION','LONG_DESCRIPTION','MANF_NAME','CATEGORY_ID'])
data.head()

,ID,NAME,text
0,9683165,Standard,FLUTE OUTFIT ODYSSEY Office Depot
1,202859,Toner Cartridges,Oki C9200/C9400 Toner Cartridge 41515210 Magen...
2,9683166,Audio Visual Equipment,CLARINET OUTFIT ODYSSEY Office Depot
3,202858,Toner Cartridges,Oki C9200/C9400 Toner Cartridge 41515211 Cyan Oki
4,9683167,Audio Visual Equipment,ALTO SAX OUTFIT ODYSSEY Office Depot


In [31]:
data.head()

,ID,NAME,text
0,9683165,Standard,FLUTE OUTFIT ODYSSEY Office Depot
1,202859,Toner Cartridges,Oki C9200/C9400 Toner Cartridge 41515210 Magen...
2,9683166,Audio Visual Equipment,CLARINET OUTFIT ODYSSEY Office Depot
3,202858,Toner Cartridges,Oki C9200/C9400 Toner Cartridge 41515211 Cyan Oki
4,9683167,Audio Visual Equipment,ALTO SAX OUTFIT ODYSSEY Office Depot


In [33]:
#Data cleaning
default_stemmer = PorterStemmer()
default_stopwords = stopwords.words('english') # or any other list of your chose
def clean_text(text, ):

    def tokenize_text(text):
        return [w for s in sent_tokenize(text) for w in word_tokenize(s)]

    def remove_special_characters(text, characters=string.punctuation.replace('-', '')):
        tokens = tokenize_text(text)
        pattern = re.compile('[{}]'.format(re.escape(characters)))
        return ' '.join(filter(None, [pattern.sub('', t) for t in tokens]))

    def stem_text(text, stemmer=default_stemmer):
        tokens = tokenize_text(text)
        return ' '.join([stemmer.stem(t) for t in tokens])

    def remove_stopwords(text, stop_words=default_stopwords):
        tokens = [w for w in tokenize_text(text) if w not in stop_words]
        return ' '.join(tokens)

    text = re.sub(r'[0-9]+', '', text)
    text = text.lower() #lowercase
    text = stem_text(text) #stemming
    text = remove_special_characters(text) #remove punctuation and symbols
    text = remove_stopwords(text) #remove stopwords
    #text.strip(' ') # strip white spaces again?

    return text

In [34]:
data.text = data.text.apply(clean_text)
data.head()

,ID,NAME,text
0,9683165,Standard,flute outfit odyssey offic depot
1,202859,Toner Cartridges,oki cc toner cartridg magenta oki
2,9683166,Audio Visual Equipment,clarinet outfit odyssey offic depot
3,202858,Toner Cartridges,oki cc toner cartridg cyan oki
4,9683167,Audio Visual Equipment,alto sax outfit odyssey offic depot


In [35]:
back_up_products = data.loc[:,['ID','text']]
features = data.text
target = data.NAME

In [36]:
#encoding target
le = LabelEncoder()
le.fit(target)
target = le.transform(target)

In [113]:
joblib.dump(le,'labelEncoder.pkl')

['labelEncoder.pkl']

In [37]:
#First Model with Count Vectorizer
cv = CountVectorizer()
cv.fit(features)
features_cv = cv.fit_transform(features)

In [115]:
joblib.dump(cv,'vectorizer.pkl')

['vectorizer.pkl']

In [38]:
from sklearn.model_selection import train_test_split
X_train_cv, X_test_cv, y_train_cv, y_test_cv = train_test_split(features_cv, target, test_size=0.2, random_state=42,stratify=target)

In [47]:
print('Records in train: ',X_train_cv.shape[0])
print('Records in test: ',X_test_cv.shape[0])

Records in train:  21544
Records in test:  5386


In [48]:
print('Features:',X_train_cv.shape[1])

Features: 8152


In [39]:
#Naive Bayes
from sklearn.naive_bayes import MultinomialNB
mNb = MultinomialNB(fit_prior=True)
mNb.fit(X_train_cv,y_train_cv)
y_pred_cv = mNb.predict(X_test_cv)

In [40]:
from sklearn.metrics import accuracy_score
print('Accuracy with NB with prior probability:',accuracy_score(y_test_cv,y_pred_cv))

Accuracy with NB with prior probability: 0.7992944671370219


In [41]:
#Naive Bayes 2
mNb2 = MultinomialNB(fit_prior=False)
mNb2.fit(X_train_cv,y_train_cv)
y_pred_cv2 = mNb2.predict(X_test_cv)
print('Accuracy with NB without prior probability:',accuracy_score(y_test_cv,y_pred_cv2))

Accuracy with NB without prior probability: 0.818418121054586


<h1>Logistic Regression</h1>

In [42]:
#Multinomial Logistic Regression
from sklearn.linear_model import LogisticRegression
lrCV = LogisticRegression(multi_class='multinomial', solver='newton-cg').fit(X_train_cv,y_train_cv)
y_pred_lrCV = lrCV.predict(X_test_cv)
print('Accuracy with LR:',accuracy_score(y_test_cv,y_pred_lrCV))

Accuracy with LR: 0.9064240623839585


In [43]:
print('Accuracy with LR:',accuracy_score(y_test_cv,y_pred_lrCV))

Accuracy with LR: 0.9064240623839585


In [44]:
#SVM
from sklearn.svm import LinearSVC
svc = LinearSVC(C=1,penalty='l2',multi_class='ovr',class_weight='balanced',random_state=42).fit(X_train_cv,y_train_cv)
y_pred_svc = svc.predict(X_test_cv)
print('Accuracy with SVM:',accuracy_score(y_test_cv,y_pred_svc))

Accuracy with SVM: 0.8975120683252877


In [46]:
features_cv.shape[1]

8152

In [53]:
#Decision Tree
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(criterion='gini',class_weight='balanced').fit(X_train_cv,y_train_cv)
y_pred_dtg = dt.predict(X_test_cv)
print('Accuracy with Decision Tree gini',accuracy_score(y_pred_dtg,y_pred_svc))

Accuracy with Decision Tree gini 0.8982547344968437


In [66]:
#Decision Tree
max_feature = int(np.sqrt(features_cv.shape[1]))
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(criterion='entropy',class_weight='balanced',max_features='auto').fit(X_train_cv,y_train_cv)
y_pred_dte = dt.predict(X_test_cv)
print('Accuracy with Decision Tree entropy',accuracy_score(y_pred_dte,y_pred_svc))

Accuracy with Decision Tree entropy 0.84478277014482


In [72]:
#Random Forest
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=400,oob_score=True,n_jobs=-1,random_state=42,class_weight='balanced').fit(X_train_cv,y_train_cv)
y_pred_rf = rf.predict(X_test_cv)
print('Accuracy with RF:',accuracy_score(y_test_cv,y_pred_rf))

Accuracy with RF: 0.9066097289268474


In [75]:
#
idf = TfidfVectorizer()
features_idf = idf.fit_transform(features)

In [77]:
from sklearn.externals import joblib

joblib.dump(lrCV,'LRmodel.pkl')

['LRmodel.pkl']

In [50]:
data.head()

,ID,NAME,text
0,9683165,Standard,flute outfit odyssey offic depot
1,202859,Toner Cartridges,oki cc toner cartridg magenta oki
2,9683166,Audio Visual Equipment,clarinet outfit odyssey offic depot
3,202858,Toner Cartridges,oki cc toner cartridg cyan oki
4,9683167,Audio Visual Equipment,alto sax outfit odyssey offic depot


In [51]:
sample = data.sample(50)

In [52]:
sample.head()

,ID,NAME,text
18773,6765763,Printer Paper,cartridg paper gsm white pk offic depot
9890,10106658,Notebook Computers,hp probook b notebook hp
26794,10116024,Standard,aec solut ihc good slide invitrogen zyme
5196,10108306,Motherboards,msi a-g socket socket motherboard msi
5920,9788253,Standard,startech port cat usb ps kvm switch startech


In [57]:
predictFromText(sample.text.iloc[3])

C:\Users\vishnu.raju\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


'Motherboards'